In [2]:
from network import Network
from conv import Conv
from dense import Dense
from activation import Relu
from metrics import log_loss, log_loss_grad, softmax
import math
import numpy as np
import random

In [4]:
random.seed(0)
FILE = "fashion-mnist_train.csv"
with open(f"../data/{FILE}") as f:
    examples = f.read().strip().split("\n")[1:]

train, valid, test = [], [], []
valid_cutoff = .8
test_cutoff = .9

for example in examples:
    label, pixels = example.split(",", 1)
    label = int(label)
    if label > 4:
        continue
    pixels = [int(p) for p in pixels.split(",")]
    data = np.asarray(pixels, dtype="int32")
    data = data.reshape((1,28,28))
    # Scale values from -1 to 1
    data = ((data / 255) - .5) / .5
    target = np.zeros((1, 5))
    target[0,label] = 1
    row = (data, target, )

    split = random.random()
    if split > test_cutoff:
        test.append(row)
    elif split > valid_cutoff:
        valid.append(row)
    else:
        train.append(row)

In [5]:
class TestNet(Network):
    def __init__(self):
        self.conv1 = Conv(input_channels=1, output_channels=3, kernel_x=3, kernel_y=3)
        self.conv2 = Conv(input_channels=3, output_channels=1, kernel_x=3, kernel_y=3)
        self.dense = Dense(input_size=24 ** 2, output_size=5, activation=False)
        self.last_input = None

        super().__init__()

    def forward(self, x):
        self.last_input = x.copy()
        x = self.conv1.forward(x)
        x = self.conv2.forward(x)
        x = x.reshape(1, math.prod(x.shape))
        x = self.dense.forward(x)
        return x

    def backward(self, grad, lr):
        prev_hidden = self.conv2.hidden.copy().reshape(1, math.prod(self.conv2.hidden.shape))
        grad = self.dense.backward(grad, lr, prev_hidden)
        grad = self.conv2.backward(grad, lr, self.conv1.hidden)
        grad = self.conv1.backward(grad, lr, self.last_input)

In [6]:
net = TestNet()
lr = 5e-4
epochs = 20

for epoch in range(epochs):
    epoch_loss = 0
    for i, img in enumerate(train):
        image, target = img
        batch = net.forward(image)

        grad = log_loss_grad(softmax(batch), target)
        epoch_loss += log_loss(softmax(batch), target)
        net.backward(grad, lr)

    print(f"Epoch {epoch} loss: {epoch_loss / len(train)}")
    match = np.zeros(len(valid))
    for i, img in enumerate(valid):
        image, target = img
        valid_pred = net.forward(image)
        match[i] = np.argmax(valid_pred) == np.argmax(target)

    print(f"Epoch {epoch} valid accuracy: {match.sum() / match.shape[0]}")

Epoch 0 loss: 0.9677727749432533
Epoch 0 valid accuracy: 0.7618570474281897
Epoch 1 loss: 0.6457135907788792
Epoch 1 valid accuracy: 0.7872411489645958
Epoch 2 loss: 0.5775221177845649
Epoch 2 valid accuracy: 0.8059452237808952
Epoch 3 loss: 0.5424171846642093
Epoch 3 valid accuracy: 0.8152972611890448
Epoch 4 loss: 0.5203083683843547
Epoch 4 valid accuracy: 0.822311289245157
Epoch 5 loss: 0.5048624050760291
Epoch 5 valid accuracy: 0.8293253173012692
Epoch 6 loss: 0.4932803514556203
Epoch 6 valid accuracy: 0.8326653306613226
Epoch 7 loss: 0.4841426057364258
Epoch 7 valid accuracy: 0.8346693386773547
Epoch 8 loss: 0.47666413975128563
Epoch 8 valid accuracy: 0.8406813627254509
Epoch 9 loss: 0.47038013739759077
Epoch 9 valid accuracy: 0.8450233800935204
Epoch 10 loss: 0.4649971406934237
Epoch 10 valid accuracy: 0.845691382765531
Epoch 11 loss: 0.4603175908419025
Epoch 11 valid accuracy: 0.8470273881095525
Epoch 12 loss: 0.456202102397566
Epoch 12 valid accuracy: 0.8483633934535738
Epoch 1